In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import CTS as cts
from scipy.constants import *
from scipy.signal import hilbert
from scipy import ndimage

sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")
from data_analysis_utils import find_all_zerocrossing, hl_envelopes_idx

plt.rcParams.update({'font.size': 20})
%matplotlib qt

In [2]:
freq_in = [330e9,600e9] # Hz
cl = ['r', 'g', 'b']

ne = [1e12, 1e13, 5e13]
Te = [5,5, 300]
Ti = [1,1, 100]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlabel(r"$\omega_s - \omega_i (rad/s)$")
ax.set_ylabel(r"$S(k, \omega)$")

for i in range(len(ne)):

    lambda_in = c / freq_in[0]
    omega_in = 2*np.pi*freq_in[0]

    alpha, omega_arr, omega_in, Skw= cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=10000, ions="He+")
    
    ax.plot(omega_arr-omega_in, Skw, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}", color=cl[i])

    a =cts.compute_alpha(lambda_in*100, 180, ne[i], Te[i])
    print('S integral = ',integrate.simpson(Skw, omega_arr))
    print('alpha = ', a)

    lambda_in = c / freq_in[1]
    omega_in = 2*np.pi*freq_in[1]

    alpha, omega_arr, omega_in, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=1000, ions="He+")
    ax.plot(omega_arr-omega_in, Skw, '--', color=cl[i])


# Create first legend for the data lines
first_legend = ax.legend(loc='upper right', fontsize=16, bbox_to_anchor=(1, 1))
# Add the first legend manually to the axes
ax.add_artist(first_legend)
# Add a second legend for line styles
custom_lines = [Line2D([0], [0], color='black', lw=2),
                Line2D([0], [0], color='black', lw=2, linestyle='--')]
second_legend = ax.legend(custom_lines, [f'{freq_in[0]/1e9:.0f} GHz', f'{freq_in[1]/1e9:.0f} GHz'], 
                          loc='upper left', fontsize=20, bbox_to_anchor=(0, 1))
plt.tight_layout()

# fig, ax = plt.subplots(figsize=(16, 6))
# ax.set_xlabel(r"frequency (GHz)")
# ax.set_ylabel(r"$S(k, \omega)$")

# for i in range(len(ne)):
#     alpha, omega_arr, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=5000, ions="He+")
#     freq_arr = omega_arr/(2*np.pi) *1e-9
#     ax.plot(freq_arr, Skw, '--',label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
    
# ax.legend(loc='upper left')
# ax.set_title("Helium")

In [ ]:
lambda_in = 2*np.pi*c / omega_arr

plt.figure(figsize=(8, 6))
for i in range(len(ne)):
    a_ls =cts.compute_alpha(lambda_in*100, 180, ne[i], Te[i])
    plt.plot(omega_arr*1e-9, a_ls, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")

plt.xlabel(r"frequency (GHz)")
plt.ylabel(r"$\alpha=1/k\lambda_D$")
plt.yticks(fontweight='bold')
plt.legend(loc='upper right', fontsize=16)
plt.tight_layout()

In [18]:
freq_ls = np.arange(300e9, 600e9, 10e9)
plt.figure(figsize=(8, 5))
plt.xlabel("frequency (GHz)")
plt.ylabel("$P_s/P_iL (1/cm) $")

for i in range(len(ne)):
    pr_ls = []
    for freq in freq_ls:

        lambda_in = c / freq
        alpha, omega_arr, omega_in, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=5000, ions="H+")
        pr = cts.power_ratio(ne[i], omega_arr, Skw, scattering_angle=180, L=1)
        pr_ls.append(pr)

    plt.plot(freq_ls*1e-9, pr_ls, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
plt.legend()
plt.tight_layout()

In [2]:
# Parameters
a = 5 # receiver radius in cm
f0 = 300e9 # units: Hz
n_cycles = 32
dt = 0.05 # units: ps
t0 = 5 # units: ps
n_e = 1e13  # cm^-3
B = 0.1 # Tesla

x_0 = 0 # units: meter
x_pl = 1 # units: meter
L_arr = np.arange(x_0, x_0+x_pl, 0.01) # units: meter

pr = cts.power_ratio(n_e, L=1, scattering_angle=180) * a**2/4
print(pr)

tarr, sig_in, freqs, signal_fft, x = cts.generate_thz_waveform(f0/1e12, 0.9, n_cycles, dt, t0, 500)
NT = len(tarr)

plt.figure(figsize=(8, 6))
plt.plot(tarr, sig_in)
plt.xlabel('Time (ps)')
plt.ylabel('Amplitude')
plt.tight_layout()

plt.figure(figsize=(8, 6))
plt.plot(freqs/1e9, np.abs(signal_fft))
plt.xlabel('Frequency (GHz)')
plt.ylabel('Amplitude')
plt.tight_layout()

1.9747118931834046e-12


In [4]:
signal_propagated, fft_propagated, vgarr = cts.propagate_through_dispersive_medium(len(tarr), freqs, signal_fft, n_e, 5)

plt.figure(figsize=(8, 6))
plt.plot(tarr, signal_propagated, linewidth=2, label='Propagated Signal')

plt.xlabel('Time (ps)')
plt.ylabel('E $\propto$ 1/R')
plt.tight_layout()

In [38]:
tot_wave = np.zeros_like(tarr)
for L in L_arr[1:]:
    signal_propagated, fft_propagated, vgarr = cts.propagate_through_dispersive_medium(len(tarr), freqs, signal_fft, n_e, L)
    prop_wave, _, _ = cts.propagate_through_dispersive_medium(NT, freqs, fft_propagated, n_e, L)
    tot_wave += prop_wave / L
    
wave_power = tot_wave**2 / np.max(tot_wave**2)
wave_power = ndimage.gaussian_filter1d(wave_power, 100)

In [46]:
start_idx = np.argwhere(tarr > 100)[0][0]
stop_idx = np.argwhere(tarr > 6700)[0][0]

plt.figure(figsize=(8, 6))
plt.plot(tarr[start_idx:stop_idx], tot_wave[start_idx:stop_idx], linewidth=2)
plt.xlabel('Time (ps)')
plt.ylabel('E $\propto$ 1/R')
plt.tight_layout()

plt.figure(figsize=(8, 6))
plt.plot(tarr[start_idx:stop_idx], wave_power[start_idx:stop_idx], linewidth=2)
plt.xlabel('Time (ps)')
plt.ylabel('Power')
plt.yscale('log')
plt.tight_layout()

In [36]:
plt.figure(figsize=(8, 6))
# plt.plot(tarr, signal_propagated)

for L in np.linspace(0, 1, 10):
    signal_propagated, fft_propagated, vgarr = cts.propagate_through_dispersive_medium(NT, freqs, signal_fft, n_e, L)
    
    max_idx = np.argmax(signal_propagated)
    start_idx = max_idx - 5000
    stop_idx = max_idx + 5000

    zc_arr = find_all_zerocrossing(signal_propagated[start_idx:stop_idx], direction='positive')
    # Find maximum signal value for threshold calculation
    max_signal = np.max(np.abs(signal_propagated[start_idx:stop_idx]))
    threshold = 0.1 * max_signal  # 10% of maximum
    
    # Filter zero crossings to remove those in low signal regions
    filtered_zc = []
    for zc in zc_arr:
        # Get the signal segment around this zero crossing
        pre_signal = signal_propagated[start_idx:stop_idx][:zc]
        post_signal = signal_propagated[start_idx:stop_idx][zc:]
        
        # Only keep zero crossings where signal amplitude exceeds threshold
        # both before and after the crossing point
        if len(pre_signal) > 0 and len(post_signal) > 0:
            if np.max(np.abs(pre_signal)) >= threshold and np.max(np.abs(post_signal)) >= threshold:
                filtered_zc.append(zc)
    
    # Convert to numpy array
    filtered_zc = np.array(filtered_zc)

    # plt.plot(tarr[start_idx:stop_idx], signal_propagated[start_idx:stop_idx])
    # plt.scatter(tarr[start_idx:stop_idx][filtered_zc], signal_propagated[start_idx:stop_idx][filtered_zc], color='red')
    period_ps = np.diff(tarr[start_idx:stop_idx][filtered_zc])
    omega_arr = 2*np.pi/(period_ps*1e-12)
    ang_arr = cts.faraday_rotation_angle(omega_arr, n_e*1e6, B, L)

    plt.plot(ang_arr, color='blue')


In [75]:
w0 = 2*np.pi*f0
omega_arr = np.repeat(w0, len(L_arr))
B_arr = np.linspace(0,0.1,len(L_arr))
ang_arr = cts.faraday_rotation_angle(omega_arr, n_e*1e6, B_arr, L_arr)

plt.figure(figsize=(8, 6))
plt.plot(L_arr, ang_arr, label='Forward')
plt.plot(L_arr, ang_arr*2, label='Return')
plt.legend()
plt.xlabel("Distance (m)")
plt.ylabel("Faraday rotation angle (deg)")
plt.tight_layout()

In [82]:
pwarr = wave_power[start_idx:stop_idx]
pwarr = pwarr / np.max(pwarr) * pr

L_interp = np.linspace(L_arr[0], L_arr[-1], len(pwarr))
ang_interp = np.interp(L_interp, L_arr, ang_arr*2)

sig_through = pwarr * (1 - ang_interp / 360)

plt.figure()
plt.plot(L_interp, pwarr, label='Unpolarized')
plt.plot(L_interp, sig_through, label='with polarizer')
plt.yscale('log')
plt.xlabel('Distance (m)')
plt.ylabel('Power at receiver')
plt.legend()
plt.tight_layout()
plt.show()